In [8]:
! /usr/local/Cellar/jupyterlab/3.0.1/libexec/bin/pip3.9 install beautifulsoup4
! /usr/local/Cellar/jupyterlab/3.0.1/libexec/bin/pip3.9 install pandas

  Using cached beautifulsoup4-4.9.3-py3-none-any.whl (115 kB)
  Using cached soupsieve-2.1-py3-none-any.whl (32 kB)
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.1/libexec/bin/python -m pip install --upgrade pip' command.
  Using cached pandas-1.2.0-cp39-cp39-macosx_10_9_x86_64.whl (10.7 MB)
  Using cached numpy-1.19.5-cp39-cp39-macosx_10_9_x86_64.whl (15.6 MB)
You should consider upgrading via the '/usr/local/Cellar/jupyterlab/3.0.1/libexec/bin/python -m pip install --upgrade pip' command.


In [22]:
#必要なライブラリをインポート
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame
import os
import random
import time
import threading

class ScrapingThreading(threading.Thread):
    def __init__(self, sc):
        self.sc = str(sc)
        threading.Thread.__init__(self)
        
    def __str__(self):
        return self.sc

    def run(self):        
#URL（賃貸住宅情報 検索結果の1ページ目）sc ex)13121 東京都足立区
# https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13123&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1
        url = 'http://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc='+ self.sc + '&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1'

        #データ取得
        result = requests.get(url)
        c = result.content

        #HTMLを元に、オブジェクトを作る
        soup = BeautifulSoup(c)

        #物件リストの部分を切り出し
        summary = soup.find("div",{'id':'js-bukkenList'})

        #ページ数を取得
        body = soup.find("body")
        pages = body.find_all("div",{'class':'pagination pagination_set-nav'})
        pages_text = str(pages)
        pages_split = pages_text.split('</a></li>\n</ol>')
        pages_split0 = pages_split[0]
        pages_split1 = pages_split0[-3:]
        pages_split2 = pages_split1.replace('>','')
        pages_split3 = int(pages_split2)

        #URLを入れるリスト
        urls = []

        #1ページ目を格納
        urls.append(url)

        #2ページ目から最後のページまでを格納
        for i in range(pages_split3-1):
            pg = str(i+2)
            url_page = url + '&pn=' + pg
            urls.append(url_page)

        name = [] #マンション名
        address = [] #住所
        locations0 = [] #立地1つ目（最寄駅/徒歩~分）
        locations1 = [] #立地2つ目（最寄駅/徒歩~分）
        locations2 = [] #立地3つ目（最寄駅/徒歩~分）
        age = [] #築年数
        height = [] #建物高さ
        floor = [] #階
        rent = [] #賃料
        admin = [] #管理費
        deposit = [] #敷
        gratuity= [] #礼
#         /保証/敷引,償却
        floor_plan = [] #間取り
        area = [] #専有面積

        #各ページで以下の動作をループ
        for url in urls:
            #物件リストを切り出し
            result = requests.get(url)
            c = result.content
            soup = BeautifulSoup(c)
            summary = soup.find("div",{'id':'js-bukkenList'})

            #マンション名、住所、立地（最寄駅/徒歩~分）、築年数、建物高さが入っているcassetteitemを全て抜き出し
            cassetteitems = summary.find_all("div",{'class':'cassetteitem'})

            #各cassetteitemsに対し、以下の動作をループ
            for i in range(len(cassetteitems)):
                #各建物から売りに出ている部屋数を取得
                tbodies = cassetteitems[i].find_all('tbody')

                #マンション名取得
                subtitle = cassetteitems[i].find_all("div",{
                    'class':'cassetteitem_content-title'})
                subtitle = str(subtitle)
                subtitle_rep = subtitle.replace(
                    '[<div class="cassetteitem_content-title">', '')
                subtitle_rep2 = subtitle_rep.replace(
                    '</div>]', '')

                #住所取得
                subaddress = cassetteitems[i].find_all("li",{
                    'class':'cassetteitem_detail-col1'})
                subaddress = str(subaddress)
                subaddress_rep = subaddress.replace(
                    '[<li class="cassetteitem_detail-col1">', '')
                subaddress_rep2 = subaddress_rep.replace(
                    '</li>]', '')

                #部屋数だけ、マンション名と住所を繰り返しリストに格納（部屋情報と数を合致させるため）
                for y in range(len(tbodies)):
                    name.append(subtitle_rep2)
                    address.append(subaddress_rep2)

                #立地を取得
                sublocations = cassetteitems[i].find_all("li",{
                    'class':'cassetteitem_detail-col2'})

                #立地は、1つ目から3つ目までを取得（4つ目以降は無視）
                for x in sublocations:
                    cols = x.find_all('div')
                    for i in range(len(cols)):
                        text = cols[i].find(text=True)
                        for y in range(len(tbodies)):
                            if i == 0:
                                locations0.append(text)
                            elif i == 1:
                                locations1.append(text)
                            elif i == 2:
                                locations2.append(text)

                #築年数と建物高さを取得
                tbodies = cassetteitems[i].find_all('tbody')
                col3 = cassetteitems[i].find_all("li",{
                    'class':'cassetteitem_detail-col3'})
                for x in col3:
                    cols = x.find_all('div')
                    for i in range(len(cols)):
                        text = cols[i].find(text=True)
                        for y in range(len(tbodies)):
                            if i == 0:
                                age.append(text)
                            else:
                                height.append(text)

            #階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積が入っているtableを全て抜き出し
            tables = summary.find_all('table')

            #各建物（table）に対して、売りに出ている部屋（row）を取得
            rows = []
            for i in range(len(tables)):
                rows.append(tables[i].find_all('tr'))

            #各部屋に対して、tableに入っているtext情報を取得し、dataリストに格納
            data = []
            for row in rows:
                for tr in row:
                    cols = tr.find_all('td')
                    for td in cols:
                        lis = td.find_all('li')
                        if len(lis) == 0:
                            text = td.find(text=True)
                            data.append(text)                 
                        for li in td.find_all('li'):
                            text = li.find(text=True)
                            data.append(text)                              


            #dataリストから、階、賃料、管理費、敷/礼/保証/敷引,償却、間取り、専有面積を順番に取り出す
            index = 0
            for item in data:
                if '階' in item:
                    floor.append(data[index])
                    rent.append(data[index+1])
                    admin.append(data[index+2])
                    deposit.append(data[index+3])
                    gratuity.append(data[index+4])
                    floor_plan.append(data[index+5])
                    area.append(data[index+6])
                index +=1

            #プログラムを4-10秒間停止する（スクレイピングマナー）
            sleep_seconds = random.randint(4, 10)
            time.sleep(sleep_seconds)
            print('next'+self.sc)

        #各リストをシリーズ化
        name = Series(name)
        address = Series(address)
        locations0 = Series(locations0)
        locations1 = Series(locations1)
        locations2 = Series(locations2)
        age = Series(age)
        height = Series(height)
        floor = Series(floor)
        rent = Series(rent)
        admin = Series(admin)
        deposit = Series(deposit)
        gratuity = Series(gratuity)
        floor_plan = Series(floor_plan)
        area = Series(area)

        #各シリーズをデータフレーム化
        suumo_df = pd.concat([name, address, locations0, locations1,
                              locations2, age, height,floor,rent,admin,deposit,gratuity,floor_plan,area],axis=1)

        #カラム名
        suumo_df.columns=['マンション名','住所','立地1','立地2','立地3','築年数','建物高さ','階','賃料','管理費',
                          '敷','礼','間取り','専有面積']

        #csvファイルとして保存
        suumo_df.to_csv('suumo_'+self.sc+'.csv', sep = '\t',encoding='utf-16')

scraping_list = ['11101', '11102', '11103', '11104', '11105', '11106', '11107', '11108' , '11109', '11110'] #さいたま市 
scraping_list.extend(['11203']) # さいたま市以外
scraping_list.extend(['13101', '13102', '13103', '13104', '13105', '13113']) # 都心
scraping_list.extend(['13106', '13107', '13108', '13118', '13121', '13122', '13123']) # 23区東部
scraping_list.extend(['13109', '13110', '13111', '13112']) # 23区南部
scraping_list.extend(['13114', '13115', '13120']) # 23区西区
scraping_list.extend(['13116', '13117', '13119']) # 23区北区


thread_list = []
for sc in scraping_list:
    if not os.path.exists('suumo_'+sc+'.csv'):
        thread = ScrapingThreading(sc)
        thread.start()
        thread_list.append(thread)
    
for thread in thread_list:
    thread.join()

next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
next13118
